using https://mrjbq7.github.io/ta-lib/

In [3]:
from ipywidgets import *
from IPython.display import display
from IPython.html import widgets

#get the json file with the quotes
filename = widgets.Text(description="symbol", width=200)
filename.value = "INFY.NS"
display(filename)


Text(value=u'INFY.NS', description=u'symbol')

In [42]:
import json
import pandas as pd
import numpy as np
import math
import talib
import sys
sys.path.append("../common")
import ohlc

#open json file and setup the data frame
with open(filename.value) as json_data:
    d = json.load(json_data)
    timestamp     = d["chart"]["result"][0]["timestamp"] 
    lowPrice      = d['chart']['result'][0]['indicators']['quote'][0]['low'] 
    highPrice     = d['chart']['result'][0]['indicators']['quote'][0]['high']
    closePrice    = d['chart']['result'][0]['indicators']['quote'][0]['close']
    openPrice     = d['chart']['result'][0]['indicators']['quote'][0]['open']
    adjustedClose = d['chart']['result'][0]['indicators']['adjclose'][0]['adjclose']
    name          = d["chart"]["result"][0]["meta"]["symbol"]
    volume        = d['chart']['result'][0]['indicators']['quote'][0]['volume']
    
candlestick_data = []
volume_data = []
for index in range(0,len(timestamp)) :
    candle = [timestamp[index] * 1000, openPrice[index], highPrice[index], lowPrice[index], closePrice[index]]
    candlestick_data.append(candle)
    volume_data.append([timestamp[index] *1000,volume[index]])
 

dataFrameInDays = pandas.DataFrame({
    'timestamp' : timestamp,
    'LOW'       : lowPrice,
    'HIGH'      : highPrice,
    'CLOSE'     : closePrice,
    'OPEN'      : openPrice
})

In [39]:
taopen = np.array(openPrice, dtype=float)
tahigh = np.array(highPrice, dtype=float)
talow = np.array(lowPrice, dtype=float)
taclose = np.array(closePrice, dtype=float)

hammer = talib.CDLHAMMER(taopen, tahigh, talow, taclose)
hanging = talib.CDLHANGINGMAN(taopen, tahigh, talow, taclose)
engulfing = talib.CDLENGULFING(taopen, tahigh, talow, taclose)
darkCloudCover = talib.CDLDARKCLOUDCOVER(taopen, tahigh, talow, taclose, 0)  #penetration is 0 here
piercing = talib.CDLENGULFING(taopen, tahigh, talow, taclose)
morningStar  = talib.CDLMORNINGSTAR(taopen, tahigh, talow, taclose,0)
eveningStar  = talib.CDLMORNINGSTAR(taopen, tahigh, talow, taclose,0)

In [45]:
sma50 = talib.MA(taclose,50)
sma100 = talib.MA(taclose,50)

sma50  = [0 if math.isnan(x) else x for x in sma50.tolist() ]
sma100 = [0 if math.isnan(x) else x for x in sma100.tolist()]

sma50

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 980.4519958496094,
 981.0069958496093,
 981.9359973144532,
 982.4109973144531,
 983.1449963378906,
 984.2539953613282,
 985.6559948730469,
 986.8729956054688,
 988.0679956054687,
 988.7699963378906,
 989.89099609375,
 991.2879956054687,
 992.6249951171875,
 993.7849951171875,
 994.5419958496094,
 995.5599951171876,
 996.1369946289062,
 996.2939953613281,
 996.9049963378907,
 997.4389953613281,
 997.0799963378906,
 997.0059973144531,
 997.1239978027344,
 996.7899975585938,
 996.6139978027344,
 996.382998046875,
 996.0689978027344,
 995.9259973144531,
 995.2909973144531,
 994.4299975585938,
 993.1219982910156,
 991.8199975585937,
 990.8699975585937,
 989.3809985351562,
 988.0089990234375,
 986.7139990234375,
 985.375,
 984.445,
 983.15,
 982.1869995117188,
 980.9699987792969,
 979.5669982910156

In [52]:
import charts

yAxis1 = dict(
    labels = dict(align= 'right', x=-3),
    title=dict(text= 'OHLC'),
    height='60%',
    lineWidth= 2
)
yAxis2 = dict(
    labels = dict(align= 'right', x=-3),
    title=dict(text= 'Volume'),
    height='35%',
    top= '65%',
    lineWidth= 2,
    offset= 0
)
options = dict(    
    title=dict(text= name + " Daily Historical"),
    chart=dict(zoomType='x'
               #, height= 500
              ),
    rangeSelector=dict( selected= 1)  ,
    tooltip=dict(split= True ),
    yAxis=[yAxis1, yAxis2],
)

cseries = dict (
    type = 'candlestick',
    name = name,
    data = candlestick_data,
    id   = 'dataseries'
    
)

vseries = dict (
    type = 'column',
    name = 'Volume',
    data = volume_data,
    yAxis= 1
    
)

s5 = dict (
    type= 'line',
    name= 'sma50',
    data= sma50
)


#hammer flags
candlestick_pattern = []
for index in range(0,len(hammer)):
    if ( hammer[index] != 0) :
        candlestick_pattern.append(dict(x=timestamp[index]*1000, title='H', text="Hammer"))
        
#hanging man flags
for index in range(0,len(hanging)):
    if ( hanging[index] != 0) :
        candlestick_pattern.append(dict(x=timestamp[index]*1000, title='h', text="Hanging Man"))

#engulfing flags      
for index in range(0,len(hanging)):
    if ( engulfing[index] > 0) :
        candlestick_pattern.append(dict(x=timestamp[index]*1000, title='E', text="Bullish Enulfing"))
    elif  ( engulfing[index] < 0) :
        candlestick_pattern.append(dict(x=timestamp[index]*1000, title='e', text="Bearish Enulfing"))
        
#dark-cloud cover
for index in range(0,len(hanging)):
    if ( darkCloudCover[index] != 0) :
        candlestick_pattern.append(dict(x=timestamp[index]*1000, title='dcc', text="Dark Cloud"))

#dark-cloud cover
for index in range(0,len(piercing)):
    if ( piercing[index] > 0) :
        candlestick_pattern.append(dict(x=timestamp[index]*1000, title='P', text="Piercing"))
        
#morning star
for index in range(0,len(morningStar)):
    if ( morningStar[index] > 0) :
        candlestick_pattern.append(dict(x=timestamp[index]*1000, title='M', text="Morning Star"))
        
#evening star
for index in range(0,len(eveningStar)):
    if ( eveningStar[index] > 0) :
        candlestick_pattern.append(dict(x=timestamp[index]*1000, title='E', text="Evening Star"))
        
aseries = dict (
    type='flags',
    data= candlestick_pattern,
    onSeries= 'dataseries',
    shape= 'circlepin',
)

#series = [ cseries, aseries,  vseries]
series = [cseries, aseries,  vseries]

charts.plot(series, show='inline', stock=True, options=options)


In [53]:
def KST(data, r1,n1,r2,n2,r3,n3,r4,n4,s):
    lightest = talib.MA(talib.ROC(data,r1),n1)
    light = talib.MA(talib.ROC(data,r2),n2)
    heavy = talib.MA(talib.ROC(data,r3),n3)
    heaviest = talib.MA(talib.ROC(data,r4),n4)

    KST_indicator = lightest + light*2 + heavy * 3 + heaviest * 4
    KST_signal = talib.MA(KST_indicator,s)
    return dict ( indicator=KST_indicator, signal = KST_signal)

kst = KST(taclose, 10,10,15,10,20,10,30,15,9)